## Create a dummy checkpoint

```
PROBLEM=genomics_binding_deepsea_gm12878
MODEL=tfti_transformer
HPARAMS_SET=tfti_transformer_debug
HPARAMS=''

USR_DIR=./tfti
DATA_DIR=./tfti/dev
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS_SET-$HPARAMS

mkdir -p $DATA_DIR $TRAIN_DIR

# Train
t2t-trainer \
  --t2t_usr_dir=$USR_DIR \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --model=$MODEL \
  --hparams_set=$HPARAMS_SET \
  --output_dir=$TRAIN_DIR \
  --hparams=$HPARAMS
```

## Dependency imports

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

import sys
sys.path.append("../tfti")
import tfti

tf.logging.set_verbosity(tf.logging.WARN)
# tf.logging.set_verbosity(tf.logging.INFO)

/Users/alexyku/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mode = tf.estimator.ModeKeys.EVAL  # set dropout to 0.0

data_dir  = "unused"  # will error out if `None` or empty-string
# to get the latest checkpoint in a directory: `tf.train.latest_checkpoint("path/to/ckpt_dir")`
# otherwise, specify the checkpoint file explicitly
ckpt_path = ("/Users/alexyku/t2t_train/genomics_binding_deepsea_gm12878/"
             "tfti_transformer-tfti_transformer_debug-/model.ckpt-1")

problem_name = "genomics_binding_deepsea_gm12878"
model_name   = "tfti_transformer"
hparams_set  = "tfti_transformer_debug"
hparams_overrides_str = ""

hparams  = trainer_lib.create_hparams(hparams_set, hparams_overrides_str, data_dir, problem_name)
model    = registry.model(model_name)(hparams, mode)
problem  = registry.problem(problem_name)
encoders = problem.get_feature_encoders()

In [27]:
tf.reset_default_graph()

# dummy data to infer
inputs_ph = tf.placeholder(tf.string, shape=[])
targets_ph = tf.placeholder(tf.int64, shape=[919])
keep_mask_ph = tf.placeholder(tf.bool, shape=[919])

# Preprocess examples.
features = {
    "inputs": inputs_ph,
    "targets": targets_ph,
    "latent_keep_mask": keep_mask_ph
}
encoded_inputs = problem.preprocess_example(features, mode, hparams)
encoded_inputs["target_space_id"] = 0  # generic id space

# Artificial batch dimension because the model demands it.
for k in ["inputs", "targets", "latents", "metrics_weights"]:
    encoded_inputs[k] =  tf.expand_dims(encoded_inputs[k], axis=0)

# Features to logits.
with tf.variable_scope(model_name):
    logits, _ = model.model_fn(encoded_inputs)
    preds = tf.nn.sigmoid(logits)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
variables_to_restore = [
    v for v in tf.contrib.slim.get_variables_to_restore()
    if "global_step" not in v.name
]
saver = tf.train.Saver(variables_to_restore)

# Initialize and restore variables.
sess = tf.InteractiveSession()
sess.run(init_op)
saver.restore(sess, ckpt_path)

def infer(inputs, targets, keep_mask):
    """Runs inference on a single example."""
    fetch = {
        "predictions": tf.reshape(preds, shape=[-1]),
        "targets": tf.reshape(features["targets"], shape=[-1]),
    }
    return sess.run(fetch, feed_dict={
        inputs_ph: inputs,
        targets_ph: targets,
        keep_mask_ph: keep_mask,
    })

In [41]:
inputs = "NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAAATTTGAAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTAATTTTAATTTTAATTTTAATTTTAATTTTAATTTGAATTTTGAATTTGAATTTGAATTTGATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGTCTCCGATTTGTACTTCCTTTCTTTCTTTCCCGTGCATTGCACCACAATGCGCTGTTCTTGGTACACGATTATTCAAAGTGCGCTACACCATAATCTACTGTTCTTTGTCTCCGCTGTGTTCCCGTGCGCTG"
targets = "0000000000000000000000000000000000000000000000000000000000001000000100000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000010001101000000000000000000000010000000000000000000000000000000000000000000000000000000"
targets = np.array(list(map(int, targets)))
keep_mask = np.zeros(targets.shape).astype(bool)

fetch = infer(inputs, targets, keep_mask)
print(f"predictions: {fetch['predictions']}")
print(f"targets: {fetch['targets']}")

predictions: [0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048
 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048
 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048
 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048
 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048 0.3146048]
targets: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
